# importando librerias

In [1]:
import pandas as pd
import numpy as np
import json
import ast
from pandas import json_normalize

In [2]:
# Leer el contenido del archivo
user_items = []
with open('../PI MLOps - STEAM/australian_users_items.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        user_items.append(ast.literal_eval(line))

# Crear el DataFrame anidado
df_items_anidado = pd.DataFrame(user_items)

In [3]:
df_items_anidado.head(2)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


In [4]:
df_items_anidado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


uso de explode

In [5]:
#usamos explode para desanidar:
df_items = df_items_anidado.explode('items', ignore_index=True)

In [6]:
df_items.head(4)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."


jsonNormalize para crear un nuevo df con columnas segun json y concat pára unirlos

In [7]:
# Usa json_normalize para expandir las columnas del objeto JSON en el DataFrame
df_items2 = json_normalize(df_items['items'])

# Combina el DataFrame original con el DataFrame resultante de json_normalize
df_items = pd.concat([df_items[['user_id']], df_items2], axis=1)

In [8]:
#quito la columna que no voy a usar, item_name lo quito por que me voy a basar en los id
df_items=df_items.drop(['playtime_2weeks','item_name'],axis=1)

In [11]:
df_items.head(2)

,user_id,item_id,playtime_forever
0,76561197970982479,10,6.0
1,76561197970982479,20,0.0


elimino los vacios y los que tienen 0 horas jugadas

In [10]:
#elimino los vacios solo si ambos campos estan vacios
df_items.dropna(subset=['item_id','playtime_forever'],how='all', inplace=True)

In [12]:
df_items = df_items[df_items['playtime_forever'] != 0]


In [13]:
df_items.head()


,user_id,item_id,playtime_forever
0,76561197970982479,10,6.0
2,76561197970982479,30,7.0
8,76561197970982479,300,4733.0
9,76561197970982479,240,1853.0
10,76561197970982479,3830,333.0


In [14]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3285246 entries, 0 to 5170013
Data columns (total 3 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   item_id           object 
 2   playtime_forever  float64
dtypes: float64(1), object(2)
memory usage: 100.3+ MB


In [18]:
df_items.describe()

,playtime_forever
count,3.285246e+06
mean,1.555251e+03
std,6.721032e+03
min,1.000000e+00
25%,4.400000e+01
50%,2.050000e+02
75%,8.080000e+02
max,6.427730e+05


In [19]:
df_items['playtime_forever'].max()

642773.0

guardar el nuevo csv

In [17]:
#guardo el df final
df_items.to_csv('../fastapi/itemsF.csv', index=False)